In [9]:
origional_text = """Pakistan is a beautiful country.
It has mountains, rivers, and deserts.
The people of Pakistan are friendly and hardworking.
Karachi is the largest city, while Islamabad is the capital."""


In [10]:
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [11]:
text = origional_text .lower()
text = re.sub(r'[^a-z\s]', '', text)
tokens = word_tokenize(text)
print(tokens)

['pakistan', 'is', 'a', 'beautiful', 'country', 'it', 'has', 'mountains', 'rivers', 'and', 'deserts', 'the', 'people', 'of', 'pakistan', 'are', 'friendly', 'and', 'hardworking', 'karachi', 'is', 'the', 'largest', 'city', 'while', 'islamabad', 'is', 'the', 'capital']


In [14]:
from nltk.tokenize import sent_tokenize

# Step 1 — Split into sentences
sentences = sent_tokenize(origional_text)

# Step 2 — Tokenize each sentence
tokenized_sentences = [word_tokenize(sentence.lower()) for sentence in sentences]

print(tokenized_sentences)


[['pakistan', 'is', 'a', 'beautiful', 'country', '.'], ['it', 'has', 'mountains', ',', 'rivers', ',', 'and', 'deserts', '.'], ['the', 'people', 'of', 'pakistan', 'are', 'friendly', 'and', 'hardworking', '.'], ['karachi', 'is', 'the', 'largest', 'city', ',', 'while', 'islamabad', 'is', 'the', 'capital', '.']]


In [16]:
!pip install gensim

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.9/27.9 MB 66.9 MB/s eta 0:00:00


In [17]:
from gensim.models import Word2Vec

In [20]:
# Train Word2Vec model
w2v_model = Word2Vec(tokenized_sentences, vector_size=50, window=3, min_count=1, sg=1)

# Example embedding
print(w2v_model.wv.key_to_index)

{'.': 0, 'the': 1, ',': 2, 'is': 3, 'and': 4, 'pakistan': 5, 'capital': 6, 'islamabad': 7, 'while': 8, 'city': 9, 'largest': 10, 'karachi': 11, 'hardworking': 12, 'friendly': 13, 'are': 14, 'of': 15, 'people': 16, 'deserts': 17, 'rivers': 18, 'mountains': 19, 'has': 20, 'it': 21, 'country': 22, 'beautiful': 23, 'a': 24}


In [24]:
print(w2v_model.vector_size)

50


In [21]:
print(w2v_model.wv[1])

[-0.01631119  0.00896482 -0.00828904  0.00163445  0.01697237 -0.00891939
  0.00904504 -0.0135247  -0.00709723  0.01879099 -0.00315681  0.00064119
 -0.00825248 -0.01537667 -0.00300022  0.00496796 -0.00175528  0.01107437
 -0.00550603  0.00448923  0.01091611  0.01668236 -0.00287032 -0.01836924
  0.00872761  0.00112683  0.01488191 -0.00160852 -0.00528974 -0.01751091
 -0.00175221  0.00566612  0.01082291  0.01410667 -0.0114204   0.00371449
  0.01223238 -0.0096161  -0.00622663  0.01358508  0.00326071  0.00038033
  0.00693194  0.00042717  0.0192661   0.01013025 -0.01779987 -0.01412022
  0.00178293  0.0127855 ]


In [23]:
print(w2v_model.wv["the"])

[-0.01631119  0.00896482 -0.00828904  0.00163445  0.01697237 -0.00891939
  0.00904504 -0.0135247  -0.00709723  0.01879099 -0.00315681  0.00064119
 -0.00825248 -0.01537667 -0.00300022  0.00496796 -0.00175528  0.01107437
 -0.00550603  0.00448923  0.01091611  0.01668236 -0.00287032 -0.01836924
  0.00872761  0.00112683  0.01488191 -0.00160852 -0.00528974 -0.01751091
 -0.00175221  0.00566612  0.01082291  0.01410667 -0.0114204   0.00371449
  0.01223238 -0.0096161  -0.00622663  0.01358508  0.00326071  0.00038033
  0.00693194  0.00042717  0.0192661   0.01013025 -0.01779987 -0.01412022
  0.00178293  0.0127855 ]


In [25]:
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

# Create vocabulary and mapping
vocab = list(w2v_model.wv.index_to_key)
word_to_index = {word: i for i, word in enumerate(vocab)}

# Convert text into sequences of indexes
sequence = [word_to_index[word] for word in tokens if word in word_to_index]

X = []
y = []
for i in range(1, len(sequence)):
    X.append(sequence[:i])
    y.append(sequence[i])

# Pad sequences to equal length
X = pad_sequences(X, maxlen=10, padding='pre')
y = to_categorical(y, num_classes=len(vocab))


In [26]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Embedding

model = Sequential()
model.add(Embedding(input_dim=len(vocab), output_dim=50, weights=[w2v_model.wv.vectors], trainable=False))
model.add(SimpleRNN(64, return_sequences=False))
model.add(Dense(len(vocab), activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X, y, epochs=200, verbose=0)


In [28]:
import numpy as np

def predict_next_word(seed_text):
    tokens = [word_to_index[w] for w in seed_text.lower().split() if w in word_to_index]
    tokens = pad_sequences([tokens], maxlen=10, padding='pre')
    pred = np.argmax(model.predict(tokens, verbose=0))
    return vocab[pred]

print(predict_next_word("pakistan"))


is
